# ***Multi-Class Prediction of Obesity Risk using LGBM🚀&EDA🔥 🔥***

#### ***In this updated version, I've implemented more assertive fine-tuning of LightGBM (LGBM). Through this meticulous refinement of parameters, the model demonstrates enhanced performance in both cross-validation (CV) assessments and leaderboard (LB) standings compared to its previous iteration. Notably, the CV score surpasses that of any publicly available benchmarks, showcasing its superior efficacy. However, it's worth noting that while the CV score excels, the LB score, albeit marginally lower, still places it competitively within the leaderboard rankings.***

# ***Load Python Libraries & Pakages***


In [ ]:
#basics
import numpy as np
import pandas as pd 
import seaborn as sns
import time
import matplotlib.pyplot as plt
import missingno as msno

import warnings
warnings.filterwarnings("ignore")

#preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import QuantileTransformer, quantile_transform

#feature engineering
from sklearn.feature_selection import mutual_info_classif

#transformers and pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn import set_config

#algorithms
from lightgbm import LGBMClassifier

#model evaluation
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, log_loss, auc, accuracy_score, balanced_accuracy_score
from sklearn.metrics import make_scorer, RocCurveDisplay, confusion_matrix

# Optuna and visualization tools
import optuna
from optuna.samplers import TPESampler
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice

random_state = 42


#  ***Reading the Data***

In [ ]:
# Read the data
#used for training
train = pd.read_csv('/kaggle/input/playground-series-s4e2/train.csv', index_col=[0])
submission_df = pd.read_csv('/kaggle/input/playground-series-s4e2/sample_submission.csv')
original = pd.read_csv('/kaggle/input/obesity-or-cvd-risk-classifyregressorcluster/ObesityDataSet.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e2/test.csv', index_col=[0])
original.index.names = ['id']
train = pd.concat([train, original])

#used for preliminary analysis
train_df = train.copy()
submission_df = submission_df.copy()
original_df = original.copy()
test_df = test.copy()

train_df.head()

### ***Checkin for missing values***

In [ ]:
%matplotlib inline
msno.matrix(train_df)
plt.show()

In [ ]:
missing = pd.DataFrame(train_df.isnull().sum().sort_values(ascending=False))
missing.columns = ["missing_count"]
missing = missing.loc[(missing!=0).any(axis=1)]
missing["missing_percent"] = missing[0:] / len(train_df) *100
missing.style.background_gradient('viridis')

## ***Descpriptive statistics***

In [ ]:
#numerical feature descriptive statistics

train_df.describe().T

In [ ]:
#categorical feature descriptive statistics

train_df.describe(include='object').T.sort_values(by=['unique'], ascending=False)

In [ ]:
#Target frequency

plt.figure(figsize=(8, 8))
palette_color = sns.color_palette('pastel')
explode = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
class_counts = train_df['NObeyesdad'].value_counts()

plt.pie(class_counts, labels=class_counts.index, autopct="%1.1f%%", startangle=140,
        colors=palette_color, explode=explode, wedgeprops=dict(width=1.1))

centre_circle = plt.Circle((0,0),0.30,fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.title('Target Distribution', fontsize=16)
plt.show()

## ***Data preprocessing for  Grouping features***

In [ ]:
# Just bookkeeping
feature_list = [feature for feature in train_df.columns if not feature  == "NObeyesdad"]

categorical_features = [feature for feature in train_df.columns if 
                        train_df[feature].dtype == "object" and feature  != "NObeyesdad"] 
numerical_features = [feature for feature in train_df.columns if 
                      feature not in categorical_features + ['NObeyesdad']]



#make sure no little feature left behind
assert feature_list.sort() == (categorical_features + numerical_features).sort()

# ***Exploratory Data Analysis(EDA)***

In [ ]:
fig, ax = plt.subplots(3, 3, figsize=(30, 30))
for var, subplot in zip(numerical_features, ax.flatten()):
    sns.boxplot(x='NObeyesdad', y=var, data=train_df, ax=subplot, palette='viridis')
    for ax in fig.axes:
        plt.sca(ax)
        plt.xticks(rotation=45)
    

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=(40, 20))
for var, subplot in zip(numerical_features, ax.flatten()):
    sns.histplot(x=var,  data=train_df, ax=subplot, hue='NObeyesdad', kde=True ,  palette='viridis')

In [ ]:
sns.pairplot(train_df[numerical_features + ["NObeyesdad"]].sample(n = 2000), hue="NObeyesdad", palette='viridis')

# ***Feature Engineering⚙️***

In [ ]:
y = train_df.NObeyesdad

In [ ]:
# determine the mutual information for numerical features

mutual_df = train_df[numerical_features ]

mutual_info = mutual_info_classif(mutual_df, y, random_state=random_state)

mutual_info = pd.Series(mutual_info)
mutual_info.index = mutual_df.columns
mutual_info = pd.DataFrame(mutual_info.sort_values(ascending=False), columns = ["Numerical_Feature_MI"] )
mutual_info.style.background_gradient("cool")


In [ ]:
mutual_df_categorical = train_df[categorical_features]
#categorical features must be encoded to get mutual information
for colname in mutual_df_categorical:
    mutual_df_categorical[colname], _ = mutual_df_categorical[colname].factorize()
mutual_info = mutual_info_classif(mutual_df_categorical, y, random_state=1)

mutual_info = pd.Series(mutual_info)
mutual_info.index = mutual_df_categorical.columns
pd.DataFrame(mutual_info.sort_values(ascending=False), columns = ["Categorical_Feature_MI"] ).style.background_gradient("cool")


# ***Preprocessing***

In [ ]:
# A custom pipeline for feature creation
class FeatureCreator(BaseEstimator, TransformerMixin):
    def __init__(self, add_attributes=True):
        self.add_attributes = add_attributes
        self.feature_names = []  
        self.original_features = []
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        if self.add_attributes:
            X_copy = X.copy()
            self.original_features = X.columns

            X_copy["BMI"] = X_copy["Weight"]  / X_copy["Height"]**2
            X_copy["healty_intake"] = X_copy["FCVC"]  * X_copy["CH2O"]
            X_copy["extra_cal_intake"] = (X_copy["CALC"] == 'Frequently').astype('int64') + (X_copy["FAVC"] == "Yes").astype('int64') 
            X_copy["meal_activity_ratio"] = X_copy["NCP"]  / (X_copy["FAF"] + 0.0001)
            X_copy["health_awareness"] = (X_copy["SCC"] == 'yes').astype('int64') + (X_copy["SMOKE"] == "no").astype('int64')  + (X_copy["FAVC"] == "no").astype('int64') 
            X_copy["active_transport"] = ((X_copy["MTRANS"] == 'Walking') | (X_copy["MTRANS"] == "Bike")).astype('int64')

            # Detect new feature names dynamically
            self.feature_names = X_copy.columns

            return X_copy[self.feature_names]
        else:
            return X

    def get_feature_names_out(self, input_features=None):
        if self.add_attributes:
            return self.feature_names
        else:
            return self.original_features

In [ ]:
#not used
creator = FeatureCreator()

In [ ]:
#categorical feature encoding
encoder  = ColumnTransformer(remainder='passthrough',
    transformers=[
        ('encoder', OneHotEncoder(handle_unknown = 'ignore'), categorical_features),
    ])

encoder

In [ ]:
# final preprocessing pipeline 
preprocessor =Pipeline([
    #('creator', creator),
    ("encoder", encoder)
])

preprocessor

# ***Modeling and hyperparameter tuning***

In [ ]:
train=train.replace({'Insufficient_Weight': 0,
                           'Normal_Weight': 1,
                           'Obesity_Type_I': 2,
                           'Obesity_Type_II': 3,
                           'Obesity_Type_III': 4,
                           'Overweight_Level_I': 5,
                           'Overweight_Level_II': 6})

In [ ]:
#seperate target
y = train_df['NObeyesdad']

train_df = train_df.drop(['NObeyesdad'], axis=1)

In [ ]:
#create train-test split. 
X_train, X_test, y_train, y_test = train_test_split(train_df, y, 
                                                    test_size=0.2,
                                                    random_state=random_state,
                                                    shuffle=True,
                                                    stratify=y)


print(X_train.shape, X_test.shape)

#### if you want Stepwise tunning with LightGBMTunerCV uncomment this


In [ ]:

#X_train = preprocessor.fit_transform(X_train)
#import optuna.integration.lightgbm as lgb

#from lightgbm import early_stopping
#from lightgbm import log_evaluation

#if __name__ == "__main__":

#    dtrain = lgb.Dataset(X_train, label=y_train)
#    params = {
#        "objective": "multiclass",
#        "metric": "multi_logloss",
#        "verbosity": -1,
#        "boosting_type": "gbdt",
#        "random_state": random_state,
#        "num_class": 7,
#        'learning_rate': 0.02
#    }

#    tuner = lgb.LightGBMTunerCV(
#        params,
#        dtrain,
#        folds=StratifiedKFold(n_splits=5),
#        callbacks=[early_stopping(20), log_evaluation(20)],
#    )

#    tuner.run()

#    print("Best score:", tuner.best_score)
#    best_params = tuner.best_params
#    print("Best params:", best_params)
#    print("  Params: ")
#    for key, value in best_params.items():
#        print("    {}: {}".format(key, value))

In [ ]:
#Fine tunning for learning rate

#def create_pipeline(trial):
#    params = {
#        "objective": "multiclass",
#        "metric": "multi_logloss",
#        "verbosity": -1,
#        "boosting_type": "gbdt",
#        "random_state": random_state,
#        "num_class": 7,
#        "learning_rate" :  trial.suggest_float("learning_rate", 0.01, 0.03 , log=True),
#        'n_estimators': 494,
#        'feature_pre_filter': False,
#        'lambda_l1': 1.2149501037669967e-07,
#        'lambda_l2': 0.9230890143196759,
#        'num_leaves': 31,
#        'feature_fraction': 0.5,
#        'bagging_fraction': 0.5523862448863431,
#        'bagging_freq': 4,
#        'min_child_samples': 20}
    
        
    
#    classifier = lgb.LGBMClassifier(**params)
    
#    return Pipeline([
#        ('preprocessor', preprocessor),
#        ('classifier', classifier)
#    ])

# Objective function for optimization
#def objective(trial):
#    pipeline = create_pipeline(trial)
    
    # Use cross-validation to evaluate the performance
#    ss = StratifiedKFold(n_splits=5)
#    score = cross_val_score(pipeline, X_train, y_train, n_jobs=-1, cv=ss)
    
    # Optimize for mean cross-validated accuracy
#    mean_score = np.mean(score)
    
    # Report intermediate result to Optuna for pruning
#    trial.report(mean_score, step=trial.number)
    
    # Handle pruning based on the current intermediate result
#    if trial.should_prune():
#        raise optuna.TrialPruned()
    
#    return mean_score

# Optuna optimization with pruning

#def optimize():
#    study = optuna.create_study(direction='maximize', pruner=optuna.pruners.HyperbandPruner())
    
    # Set the number of optimization trials
#    n_trials = 200
    
    # Optimize
#    study.optimize(objective, n_trials=n_trials, n_jobs=1)  # Set n_jobs to 1 to ensure proper pruning
    
#    print(f'Best trial: {study.best_trial.params}')
#    print(f'Best accuracy: {study.best_value}')

#if __name__ == '__main__':
#    optimize()

In [ ]:
# best params from Optuna
lgbm_params = {"objective": "multiclass",
               "metric": "multi_logloss",
               "verbosity": -1,
               "boosting_type": "gbdt",
               "random_state": random_state,
               "num_class": 7,
               "learning_rate" :  0.01386432121252535,
               'n_estimators': 494,
               'feature_pre_filter': False,
               'lambda_l1': 1.2149501037669967e-07,
               'lambda_l2': 0.9230890143196759,
               'num_leaves': 31,
               'feature_fraction': 0.5,
               'bagging_fraction': 0.5523862448863431,
               'bagging_freq': 4,
               'min_child_samples': 20}

In [ ]:
%%time

lgbm_tunned = LGBMClassifier(**lgbm_params)

pipe_lgbm = Pipeline([
    
    ('preprocessor', preprocessor),
    ("lgbm_tunned", lgbm_tunned)
])


# Cross-validation scores
cv = StratifiedKFold(n_splits=5)

cv_scores = cross_val_score(pipe_lgbm, X_train, y_train, cv=cv, scoring='accuracy')  # 5-fold cross-validation
print("Mean CV Score:", np.mean(cv_scores))

#Test score
pipe_lgbm = pipe_lgbm.fit(X_train, y_train)
preds_test = pipe_lgbm.predict(X_test)
print("Test Score:", accuracy_score(y_test, preds_test))

# ***Explainable ML & Feature importances***

In [ ]:
X_train_explain =preprocessor.fit_transform(X_train)
X_train_explain = pd.DataFrame(X_train_explain, columns=preprocessor.get_feature_names_out())
X_test_explain = preprocessor.transform(X_test)
X_test_explain = pd.DataFrame(X_test_explain, columns=preprocessor.get_feature_names_out())

In [ ]:
#plot feature importance and summarize model performance
from yellowbrick.features import FeatureImportances
from yellowbrick.classifier import ConfusionMatrix, ClassificationReport, ROCAUC, DiscriminationThreshold

fig, axes = plt.subplots(2, 2, figsize=(15, 15))

model = lgbm_tunned
model.importance_type = 'gain'

visualgrid = [
    FeatureImportances(model,  ax=axes[0][0], colormap= 'winter'),
    ConfusionMatrix(model, ax=axes[0][1], cmap= 'GnBu'),
    ClassificationReport(model, ax=axes[1][0], cmap= 'GnBu'),
    ROCAUC(model, ax=axes[1][1]),
]

for viz in visualgrid:
    viz.fit(X_train_explain, y_train)
    viz.score(X_test_explain, y_test)
    viz.finalize()

plt.show()

Shap is another tool for explainability. 

In [ ]:

import shap

model = lgbm_tunned
model.fit(X_train_explain, y_train)

explainer = shap.TreeExplainer(model, data=X_train_explain, feature_perturbation='interventional')
#interventional argument breaks feature dependencies and provides better estimate

#It takes time just take some sample
X_test_explain_sampled = X_test_explain.sample(frac=0.1, random_state=random_state)

shap_values = explainer.shap_values(X_test_explain_sampled, check_additivity=False )

# Summary plot of feature importance
shap.summary_plot(shap_values, X_test_explain_sampled)

# ***Submission***

In [ ]:
pipe_lgbm = pipe_lgbm.fit(train_df, y)
preds = pipe_lgbm.predict(test)

In [ ]:
output = pd.DataFrame({'id': test.index,
                       'NObeyesdad': preds})

#revert clas numbers to names this is how submission is asked
output= output.replace({0: 'Insufficient_Weight',
                        1: 'Normal_Weight',
                        2: 'Obesity_Type_I',
                        3: 'Obesity_Type_II',
                        4: 'Obesity_Type_III',
                        5: 'Overweight_Level_I',
                        6: 'Overweight_Level_II'})

In [ ]:
output.to_csv('submission.csv', index=False)

output.head()